In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
    "X": "something"
}
step_params={
    "X": "something_else"
}
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    
    outputs = 
    [
        { ENTITY_NAME: "bento_service" }
    ],

)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# Train
from sklearn import svm
from sklearn import datasets
from bentoservice import IrisClassifier

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

# Create a iris classifier service with the newly trained model
iris_classifier_service = IrisClassifier()
iris_classifier_service.pack("model", clf)

In [ ]:
# Test model API
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice

# Test invoking BentoService instance
iris_classifier_service.predict([[5.1, 3.5, 1.4, 0.2]])

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(iris_classifier_service)

# Stop the dev model server
stop_dev_bentoservice(iris_classifier_service)

In [ ]:
# Serve model
from sinara.bentoml import save_bentoservice

outputs = substep.outputs()

# Save the entire prediction service to a BentoML bundle
save_bentoservice(iris_classifier_service, (outputs, 'bento_service'))